In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [16]:
train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')
train_extra = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv')

In [17]:
train.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [43]:
train_data = pd.concat([train, train_extra],axis=0,ignore_index=True)
train_data.shape

(3994318, 11)

In [44]:
%%time
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994318 entries, 0 to 3994317
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Brand                 object 
 2   Material              object 
 3   Size                  object 
 4   Compartments          float64
 5   Laptop Compartment    object 
 6   Waterproof            object 
 7   Style                 object 
 8   Color                 object 
 9   Weight Capacity (kg)  float64
 10  Price                 float64
dtypes: float64(3), int64(1), object(7)
memory usage: 335.2+ MB
CPU times: user 7.19 ms, sys: 2 ms, total: 9.2 ms
Wall time: 8.62 ms


In [21]:
# Lebale encoding
from sklearn.preprocessing import LabelEncoder
columns = ['Brand','Material','Size','Laptop Compartment','Waterproof','Style','Color',]

# Train data encoding
def data_encoding(data, encoders):
    for col in columns:
        if col not in encoders:
            encoders[col] = LabelEncoder()
        le = encoders[col]
        data[col] = le.fit_transform(data[col])
    data['Weight Capacity (kg)'] = data['Weight Capacity (kg)'].fillna(data['Weight Capacity (kg)'].mean())


In [22]:
encoders = {}
data_encoding(train_data, encoders)
data_encoding(test, encoders)

In [23]:
train_data.isnull().sum()

id                      0
Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64

In [24]:
test.isnull().sum()

id                      0
Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
dtype: int64

In [31]:
train_data.describe()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
count,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06,3.994318e+06
mean,2.182137e+06,2.098836e+00,1.624305e+00,1.025793e+00,5.434740e+00,5.432725e-01,5.402306e-01,1.061032e+00,2.635639e+00,1.801042e+01,8.136217e+01
std,1.178058e+06,1.500988e+00,1.176810e+00,8.521525e-01,2.893043e+00,5.454027e-01,5.437009e-01,8.611790e-01,1.782202e+00,6.972391e+00,3.893868e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.500000e+01
25%,1.198579e+06,1.000000e+00,1.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.207231e+01,4.747002e+01
50%,2.197158e+06,2.000000e+00,2.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.805142e+01,8.098495e+01
75%,3.195738e+06,3.000000e+00,3.000000e+00,2.000000e+00,8.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00,2.398552e+01,1.148550e+02
max,4.194317e+06,5.000000e+00,4.000000e+00,3.000000e+00,1.000000e+01,2.000000e+00,2.000000e+00,3.000000e+00,6.000000e+00,3.000000e+01,1.500000e+02


In [37]:
# Train data selection
y_feature = ['Price']
x_feature = list(test.columns)
x_train_data = train_data[x_feature]
y_train_data = train_data[y_feature]


# Data Scalling
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
x_scale_train = scaler.fit_transform(x_train_data)
scale_test = scaler.fit_transform(test)

In [40]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import LinearRegression

# x_train, x_test, y_train, y_test = train_test_split(
#     x_scale_train, y_train_data, test_size=0.2, random_state=42)

# lmodel = LinearRegression()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# plt.scatter(y_test, y_pred)
# plt.xlabel("True Sale Price")
# plt.ylabel("Predicted Sale Price")
# plt.title(f"{name}: True vs. Predicted Sale Price")
# plt.show()


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Unknown label type: 'continuous'

In [28]:
# Old Method to run mutiple models
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import LinearSVR, SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt

# Models list
models = [
    ('LiR', LinearRegression()),
    ('LoR', LogisticRegression()),
    ('KNN', KNeighborsRegressor(n_neighbors=8)),
    ('LSV', LinearSVR()),
    ('SVR', SVR()),
    ('RFR', RandomForestRegressor()),
    ('GBR', GradientBoostingRegressor()),
    ('LGB', LGBMRegressor()),
    ('CBR', CatBoostRegressor())
]

# Feature scaling
# scacler = StandardScaler()
# # x_scaled = scacler.fit_transform(x_train_data)
# predict_x_test =  scacler.fit_transform(X_test_data)
test_data = test
x_scaled = x_train_data
predict_x_test = test
# predict_x_test = X_test_data


x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y_train_data, test_size=0.2, random_state=42)

max_output_score, max_output_df = float('-inf'), None

for name, model in models:
    try:
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(name)
        print(f"Mean Squared Error (MSE): {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
        print(f"Mean Absolute Error (MAE): {mae:.2f}")
        print(f"R-squared (R2): {r2:.2f}")
        plt.scatter(y_test, y_pred)
        plt.xlabel("True Sale Price")
        plt.ylabel("Predicted Sale Price")
        plt.title(f"{name}: True vs. Predicted Sale Price")
        plt.show()
        
        output = model.predict(predict_x_test)
        arr = test_data.loc[:,'id']
        output_df = pd.DataFrame({'id':arr, 'yield': output.flatten()})
        output_df.to_csv(f'/kaggle/working/{name}-output.csv', encoding='utf-8', index=False)
        
        # If it is most optimise solution then store in submission.csv
        if max_output_score < r2:
            max_output_score, max_output_df = r2, output_df.copy()
    except Exception as ex:
        print('Faild = ', name)
        print(ex)

print('Final output store in submission.csv')
max_output_df.to_csv(f'/kaggle/working/submission.csv', encoding='utf-8', index=False)

Faild =  LoR
Unknown label type: 'continuous'
Final output store in submission.csv


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AttributeError: 'NoneType' object has no attribute 'to_csv'